In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional 
from keras.optimizers import RMSprop

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence

from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import re

import gensim
#from pymystem3 import Mystem
#from stanfordcorenlp import StanfordCoreNLP
import spacy_udpipe

Using TensorFlow backend.
unable to import 'smart_open.gcs', disabling that module


In [2]:
df = pd.read_csv('task2_en_training.tsv',delimiter='\t')
df.head() 

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make thin..."
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made ch...
4,345567138376994816,138795534,0,have decided to skip my #humira shot today. my...


In [3]:
spacy_udpipe.download("en")
nlp = spacy_udpipe.load("en")
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True) 
#model.vocab 

Already downloaded a model for the 'en' language


In [4]:
def lemmatize_with_postag(text = """"Wikipedia is a free online encyclopedia, created and 
                          edited by volunteers around the world.""",
                          nlp=nlp):
    doc = nlp(text)
    tagged = []
    for token in doc:
        lemma = token.lemma_
        pos = token.pos_
        #pos = pos.split('=')[0].strip()
        if pos not in ["PUNCT"]: #["VERB", "NUM", "ADV", "NOUN", "ADJ", "AUX", "PRON", "ADP", "DET", "ADV", "INTJ", "PROPN"]:
            tagged.append(lemma.lower() + '_' + pos)
        #else : print(pos)
    return tagged

In [5]:
X_train = df['tweet']
y_train = df['class']

Xy = pd.concat([X_train, y_train], axis=1)

# paramters of the text praprations
n_items_for_train = 50
n_items_for_test  = 100
max_words         = 500 

# n_items_for_train твитов для тренировки с индексом класса 1
df_1 = Xy[Xy['class'] == 1].iloc[:n_items_for_train] 
# n_items_for_train твитов для тренировки с индексом класса 0
df_0 = Xy[Xy['class'] == 0].iloc[:n_items_for_train] 

Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_train = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)



# n_items_for_test твитов для теста с индексом класса 1
df_1 = Xy[Xy['class'] == 1].iloc[n_items_for_train:n_items_for_train+n_items_for_test]
# n_items_for_test твитов для теста с индексом класса 0
df_0 = Xy[Xy['class'] == 0].iloc[n_items_for_train:n_items_for_train+n_items_for_test]

Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_test = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)

print(Xy_train.shape)
print(Xy_test.shape)
#Xy_train, Xy_test

(100, 2)
(200, 2)


In [6]:
Xy_test

,tweet,class
74,"[no_DET, lamotrigine_NOUN, study_NOUN, have_AU...",0
106,"[@cotoncity_mayor_NOUN, i_PRON, get_VERB, traz...",0
50,"[so_ADV, 30_NUM, -_SYM, 60_NUM, mg_NOUN, of_AD...",0
1169,"[i_PRON, seem_VERB, to_PART, need_VERB, to_PAR...",1
1191,"[rt_NOUN, @silkius_NOUN, @ouch_uk_NOUN, do_AUX...",1
...,...,...
127,"[suck_VERB, effexor_NOUN, 's_PART, dick_NOUN, ...",0
131,"[parent_NOUN, 452/1000_NUM, &_CCONJ, amp_NOUN,...",0
1424,"[day_NOUN, 4_NUM, the_DET, initial_ADJ, lamotr...",1
1232,"[rt_NOUN, @vyvanseswag_NOUN, i_PRON, do_AUX, n...",1


In [7]:
# replcade as parameter in the begining
#max_words = 2000 # ограничение вокуабуляра

maxima = 0
for ar in Xy_train['tweet'].to_list():
    if len(ar)>maxima: maxima = len(ar)
for ar in Xy_test['tweet']:
    if len(ar)>maxima: maxima = len(ar)
max_len = maxima + 1
print(f'max_len = {max_len}')

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xy_train["tweet"]) #fit on train

# exctrax features from train
sequences_train = tok.texts_to_sequences(Xy_train["tweet"])
sequences_matrix_train = sequence.pad_sequences(sequences_train, maxlen=max_len)

# exctrax features from test
sequences_test = tok.texts_to_sequences(Xy_test["tweet"])
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

max_len = 33


In [8]:
sequences_matrix_test.shape

(200, 33)

In [9]:
def fix_word_key(string):
    upper_part = (re.findall('_.*',string))[0].upper()
    result = re.sub(r'_.*', upper_part, string)
    #print(f'was {string} -> became {result}')
    return result

word_index = tok.word_index
#print('Found %s unique tokens' % len(word_index))

wi1 = len(word_index)

# fix tokenizer problem
for key in word_index.keys():
    fixed_key = fix_word_key(key)
    word_index[fixed_key] = word_index.pop(key)
    
#print('Found %s unique tokens' % len(word_index))
wi2 = len(word_index)

if wi1 != wi2:
    print()
    error = 'lenght of word_index was changed!'
    print(error.upper())
    raise ValueError
else:
    print('Everything is OK')
    print(len(word_index))

Everything is OK
839


In [10]:
# делаем Embedding на основе w2v модели

nb_words = min(max_words, len(word_index))+1 # проверяем где меньше, в нашем датасете или в токенайзере.
EMBEDDING_DIM = 300 # размерность векторов в нашей модели w2v

print(f'number of words = {nb_words}')

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
print(embedding_matrix.shape)

counter = 0
for word, i in word_index.items():
    if i >= nb_words: continue
    if word in w2v_model.vocab:
        print(word, i)
        embedding_matrix[i] = w2v_model[word]
        #break
#     if counter > nb_words: break
#     counter += 1


print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
#pd.DataFrame(embedding_matrix)

number of words = 501
(501, 300)
take_VERB 10
day_NOUN 23
make_VERB 24
rt_NOUN 25
help_VERB 29
still_ADV 31
give_VERB 33
get_VERB 34
sleep_NOUN 35
try_VERB 37
weight_NOUN 38
diary_NOUN 40
go_VERB 41
wake_VERB 42
need_VERB 46
fuck_VERB 47
mg_NOUN 48
side_NOUN 52
effect_NOUN 53
leg_NOUN 58
feel_VERB 59
night_NOUN 60
know_VERB 62
17_NUM 68
depression_NOUN 70
cause_VERB 73
gain_NOUN 76
much_ADV 77
good_ADJ 81
week_NOUN 82
think_VERB 84
dr_NOUN 88
time_NOUN 90
16_NUM 91
like_VERB 92
gain_VERB 93
thing_NOUN 95
hour_NOUN 97
really_ADV 99
start_VERB 100
dream_NOUN 101
hip_NOUN 102
ach_NOUN 103
300_NUM 104
morning_NOUN 107
vision_NOUN 108
might_AUX 109
someone_PRON 111
lozenge_NOUN 112
risk_NOUN 114
infection_NOUN 115
eat_VERB 117
always_ADV 118
like_ADP 119
never_ADV 121
pm_NOUN 123
pain_NOUN 124
neck_NOUN 125
month_NOUN 127
weird_ADJ 130
rash_NOUN 132
amp_NOUN 135
chest_NOUN 136
great_ADJ 139
stone_NOUN 140
trial_NOUN 144
nightmare_NOUN 146
nothing_PRON 148
suit_VERB 149
severe_ADJ 150
switch

In [11]:
embedding_matrix.shape

(501, 300)

# BI-LSTM (3.0)

https://stackoverflow.com/questions/44924690/keras-the-difference-between-lstm-dropout-and-lstm-recurrent-dropout

In [12]:
rate_drop_lstm  = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'


embedding_layer      = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=max_len,
                                trainable=False) 
lstm_layer           = Bidirectional(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))

# CONFIGURATION OF BI-LSTM
sequence_1_input     = Input(shape=(max_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1                   = lstm_layer(embedded_sequences_1)
merged               = Dropout(rate_drop_dense)(x1)
merged               = BatchNormalization()(merged)
merged               = Dense(32, activation=act)(merged)
merged               = Dropout(rate_drop_dense)(merged)
merged               = BatchNormalization()(merged)
preds                = Dense(1, activation='sigmoid')(merged)
#preds                = Dense(2, activation='softmax')(merged)

In [13]:
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(10, 10, rate_drop_lstm, \
        rate_drop_dense)

model = Model(inputs=[sequence_1_input], outputs=preds)
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])
model.summary()
print(STAMP, end='\n\n')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 33)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 33, 300)           150300    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0   

In [14]:
from myclass import DataGenerator

epochs = 200
batch_size = 32
patience = 20


early_stopping =EarlyStopping(monitor='val_loss', patience=patience)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

# ================================================================================= ->
# Parameters
params = {'batch_size': batch_size, #(n_items_for_train, max_len, EMBEDDING_DIM), # (number of channels, number of classes, batch size)
          'dim': max_len,
          'n_classes': 1,
          'shuffle': True}

# Datasets

np.array([[index, seq] for index, seq in enumerate(sequences_matrix_train.tolist())])
np.array([[index, seq] for index, seq in enumerate(sequences_matrix_test.tolist())])

# IDs    {'train': ['id-1', 'id-2', 'id-3'], 'validation': ['id-4']}
partition_train = sequences_matrix_train#.tolist()
partition_val   = sequences_matrix_test[:n_items_for_test//2] #.tolist()[:n_items_for_test//2]
partition_test  = sequences_matrix_test[n_items_for_test//2:] #.tolist()[n_items_for_test//2:]

# Labels {'id-1': 0, 'id-2': 1, 'id-3': 2, 'id-4': 1}
labels_train = np.array(Xy_train["class"]).tolist()
labels_val = np.array(Xy_test["class"]).tolist()[:n_items_for_test//2]
#print(labels_train)
#print(partition_val)
# Generators
training_generator   = DataGenerator(partition_train, labels_train, **params)
validation_generator = DataGenerator(partition_val, labels_val, **params)

In [15]:
# partition_train.shape
# len(training_generator)
# np.empty((32, 2, 1))
# partition_train
# training_generator[1]
# np.empty((32), dtype=int)
X, y = training_generator[0]
# print(y)
#validation_generator[0]
X.shape
y.shape

(32,)

In [16]:
# Train model on dataset
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              #steps_per_epoch=100,
                              #use_multiprocessing=True, workers=4,
                              epochs=epochs, shuffle=True,
                              verbose=2,
                              callbacks=[early_stopping, model_checkpoint])
# ================================================================================= <-

# history = model.fit(sequences_matrix_train, Xy_train["class"], 
#                  validation_data=(sequences_matrix_test[:n_items_for_test//2], 
#                                   Xy_test["class"][:n_items_for_test//2]), 
#                  epochs=epochs, batch_size=batch_size, shuffle=True,
#                  callbacks=[early_stopping, model_checkpoint])

Epoch 1/200
 - 2s - loss: 0.9810 - acc: 0.4792 - val_loss: 0.6943 - val_acc: 0.4375
Epoch 2/200
 - 0s - loss: 0.7543 - acc: 0.6250 - val_loss: 0.6893 - val_acc: 0.5312
Epoch 3/200
 - 0s - loss: 0.6610 - acc: 0.6667 - val_loss: 0.6986 - val_acc: 0.3750
Epoch 4/200
 - 0s - loss: 0.5439 - acc: 0.7500 - val_loss: 0.6959 - val_acc: 0.4062
Epoch 5/200
 - 0s - loss: 0.4460 - acc: 0.7917 - val_loss: 0.6913 - val_acc: 0.5000
Epoch 6/200
 - 0s - loss: 0.5603 - acc: 0.6979 - val_loss: 0.6910 - val_acc: 0.5938
Epoch 7/200
 - 0s - loss: 0.4941 - acc: 0.7292 - val_loss: 0.6939 - val_acc: 0.5000
Epoch 8/200
 - 0s - loss: 0.4012 - acc: 0.8229 - val_loss: 0.6912 - val_acc: 0.5312
Epoch 9/200
 - 0s - loss: 0.4627 - acc: 0.7500 - val_loss: 0.6914 - val_acc: 0.5000
Epoch 10/200
 - 0s - loss: 0.4317 - acc: 0.7917 - val_loss: 0.6924 - val_acc: 0.5625
Epoch 11/200
 - 0s - loss: 0.3889 - acc: 0.8542 - val_loss: 0.6876 - val_acc: 0.5938
Epoch 12/200
 - 0s - loss: 0.3880 - acc: 0.7917 - val_loss: 0.6939 - val_a

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParaEMBEDDING_DIMms["figure.figsize"] = (10, 8) 

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(dgclasshistory.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'rcParaEMBEDDING_DIMms'

In [ ]:
pred = model.predict(sequences_matrix_test[n_items_for_test//2:])
pred.shape
bin_pred, real = [], np.array(Xy_test["class"]).tolist()[n_items_for_test//2:]
for i in range(len(pred)):
    #print(f'pred = {int(round(float(pred[i][0]), 0))},\t real = {real[i]}')
    bin_pred.append(int(round(float(pred[i][0]), 0))) 
#np.array(Xy_test["class"]).tolist()
bin_pred, real
from sklearn.metrics import classification_report
print(classification_report(real, bin_pred))

In [ ]:
pd.DataFrame(confusion_matrix(real, bin_pred), columns=['Pos', 'Neg'])